In [1]:
import customtkinter as ctk
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from catboost import CatBoostRegressor
from joblib import load

**Получение данных для примеров графиков**

In [2]:
df = pd.read_excel('Телеметрия.xlsx')

text_Y = []
cols = list(df.columns)
text_X = list(df.head(1).values)
text_Y.append(text_X[0][-3])
text_Y.append(text_X[0][-4])

all_values = []
for i in range(180):
    all_values.append(list(df.iloc[i][['Гранулометрия 1', 'Гранулометрия 2']]))

all_values_1 = []
all_values_2 = []
for i in range(len(all_values)):
    all_values_1.append(all_values[i][0])
    all_values_2.append(all_values[i][1])

**Plot**

In [3]:
def plot(for_plot1, for_plot2, win):
    global plot_frame, plot_gr1, plot_gr2
    
    plot_frame = ctk.CTkFrame(win, width=575, height=200, fg_color='#242424')
    plot_frame.pack(pady=(0, 20))

    plot_gr1 = ctk.CTkFrame(plot_frame, width=575/2 - 40, height=200, fg_color='#242424', border_width=0.75, border_color='#8FBC8F')
    plot_gr1.pack(padx=10, side='left', fill="both", expand=True)

    plot_gr2 = ctk.CTkFrame(plot_frame, width=575/2 - 40, height=200, fg_color='#242424', border_width=0.75, border_color='#8FBC8F')
    plot_gr2.pack(padx=10, side='left', fill="both", expand=True)

    plot_gr1__ = ctk.CTkFrame(plot_gr1, width=575/2 - 38, height=150, fg_color='#242424')
    plot_gr1__.pack(padx=10, pady=10, side='left', fill="both", expand=True)

    plot_gr2__ = ctk.CTkFrame(plot_gr2, width=575/2 - 38, height=150, fg_color='#242424')
    plot_gr2__.pack(padx=10, pady=10, side='left', fill="both", expand=True)


    fig1, ax1 = plt.subplots(figsize=(2.5, 2))
    ax1.tick_params(axis='both', which='major', labelsize=8, labelcolor='white', color='white')
    ax1.set_facecolor('#242424')
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['bottom'].set_color('white')
    ax1.spines['left'].set_color('white')
    fig1.patch.set_facecolor('#242424')
    ax1.plot(for_plot1, color='orange')

    canvas1 = FigureCanvasTkAgg(fig1, master=plot_gr1__)
    canvas1.draw()
    canvas1.get_tk_widget().pack(side=ctk.TOP, fill=ctk.BOTH, expand=True)



    fig2, ax2 = plt.subplots(figsize=(2.5, 2))
    ax2.tick_params(axis='both', which='major', labelsize=8, labelcolor='white', color='white')
    ax2.set_facecolor('#242424')
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['bottom'].set_color('white')
    ax2.spines['left'].set_color('white')
    fig2.patch.set_facecolor('#242424')
    ax2.plot(for_plot2, color='orange')
    
    canvas2 = FigureCanvasTkAgg(fig2, master=plot_gr2__)
    canvas2.draw()
    canvas2.get_tk_widget().pack(side=ctk.TOP, fill=ctk.BOTH, expand=True)

**Button's**

In [4]:
def reset_search():

    for row, item in enumerate(text_X, start=1):
        for col, value in enumerate(item):
            cell = ctk.CTkLabel(
                scroll_frame,
                text='0',
                width=150,
                height=30,
                fg_color="#404040",
                corner_radius=5,
                font=("Arial", 12)
            )
            cell.grid(row=col, column=row, padx=10, pady=2, sticky='w')

    gr1.configure(text='Гранулометрия 1:     0')
    gr2.configure(text='Гранулометрия 2:     0')

    global plot_frame, plot_gr1, plot_gr2, win
    plot_frame.destroy()
    plot_gr1.destroy()
    plot_gr2.destroy()
    
    plot(0, 0, win)


def save_search():
    global fig1, fig2
    fig1.savefig('fig1.png')
    fig2.savefig('fig2.png')

**Predict**

In [5]:
def deploy(data_):
    data = pd.read_csv(data_, encoding='utf-8', delimiter=';').tail(180)

    data['Время'] = pd.to_datetime(data['Время'])
    data = data.dropna()
    data["часы"] = data["Время"].dt.hour
    data["минуты"] = data["Время"].dt.minute
    data = data.drop(columns=['Время', 'Гранулометрия 1 %'])

    data = data.drop(columns=['Мощность МПСИ 2 кВт', 'Мощность МШЦ 2 кВт', 'Ток МПСИ 2 А', 'Ток МШЦ 2 А', 'Исходное питание МПСИ 2 т/ч', 'Возврат руды МПСИ 2 т/ч', 'Общее питание МПСИ 2 т/ч', 
                        'Расход воды МПСИ 2 PV м3/ч', 'Расход воды МПСИ 2 SP м3/ч', 'Расход воды МПСИ 2 CV %', 'факт соотношение руда/вода МПСИ 2', 'Давление на подшипник МПСИ 2 загрузка Бар', 
                        'Давление на подшипник МПСИ 2 разгрузка Бар', 'Обороты насоса 2 1 %', 'Давление в ГЦ насоса 2 1 Бар', 'Расход в ГЦ насоса 2 1 м3/ч', 'Расход оборотной воды 2 м3/ч', 'Гранулометрия 2 %', 
                        'Поток 2 л/мин'])
    
    CB = CatBoostRegressor()
    CB.load_model("CatBoost_model.cbm")
    mmd = load('HistGradientBoostingRegressor.pkl')

    pred_CB = CB.predict(data)
    pred_mmd = mmd.predict(data)

    predict = (pred_CB + pred_mmd) / 2

    return predict

# Main

In [ ]:
win = ctk.CTk()
win.geometry('575x900')

win.title('GrAI')
win.iconbitmap('Logo.ico')


# --------------------------------СКРРОЛЛБАР---------------------------------------------


scroll_frame = ctk.CTkScrollableFrame(win, border_color='#8FBC8F', border_width=1, orientation='vertical', width=575, scrollbar_button_color='#8FBC8F', scrollbar_button_hover_color='#AFEEEE')
scroll_frame.pack(pady=(20, 10), padx=20, fill="y", expand=True)

for col, header in enumerate(cols):
    label = ctk.CTkLabel(
        scroll_frame,
        text=header,
        width=150,
        height=30,
        fg_color="#333333",
        corner_radius=5,
        font=("Arial", 14, "bold")
    )
    label.grid(row=col, column=0, padx=5, pady=5, sticky='w')

for row, item in enumerate(text_X, start=1):
    for col, value in enumerate(item):
        cell = ctk.CTkLabel(
            scroll_frame,
            text=value,
            width=150,
            height=30,
            fg_color="#404040",
            corner_radius=5,
            font=("Arial", 12)
        )
        cell.grid(row=col, column=row, padx=10, pady=2, sticky='w')


#-----------------------------------ПАКАЗАНИЯ ГРАНУЛОМЕТРИИ И КНОПКИ--------------------------------------------------------------


parent_btn = ctk.CTkFrame(win, width=575, height=50, fg_color='#242424')
parent_btn.pack(padx=20, pady=0)

btn_reset = ctk.CTkButton(parent_btn, text='Reset search', width=50, height=40, fg_color='#8FBC8F', command=reset_search, text_color='black', hover_color='#AFEEEE')
btn_reset.place(x=175, y=0)
btn_save = ctk.CTkButton(parent_btn, text='Save search', width=50, height=40, fg_color='#8FBC8F', command=save_search, text_color='black', hover_color='#AFEEEE')
btn_save.place(x=270, y=0)

parent_res_frame = ctk.CTkFrame(win, width=575, height=120, fg_color='#242424')
parent_res_frame.pack(padx=20, pady=10)

first_side = ctk.CTkFrame(parent_res_frame, width=555/2, height=60, fg_color='#242424')
first_side.grid(row=0, column=0)
second_side = ctk.CTkFrame(parent_res_frame, width=555/2, height=60, fg_color='#242424')
second_side.grid(row=0, column=1)

gr1 = ctk.CTkLabel(first_side, text=f'Гранулометрия 1:\n\n{text_Y[0]}', font=('Arial', 16))
gr1.place(x=60, y=0)
gr2 = ctk.CTkLabel(second_side, text=f'Гранулометрия 2:\n\n{text_Y[1]}', font=('Arial', 16))
gr2.place(x=60, y=0)


#-----------------------------------ГРАФИКИ--------------------------------------------------


plot(deploy('granulometry_v2.csv'), all_values_2, win)

win.mainloop()